In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import scvelo as scv
import loompy
import pandas as pd
import numpy as np
import os
import scanpy as sc
import scipy.stats as stats
from io import StringIO
from sklearn import linear_model
import seaborn as sns
 
%matplotlib inline

In [ ]:
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')
# for beautified visualization

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

## input sample path

First lets find the directory that we are working in

**Note:  SigsDir must be changed depending on the user to allow for proper use of this script**

In [ ]:
os.getcwd()

In [ ]:
#set directory to where signature gene lists locates
SigsDir="/mnt/533ee9c3-18c0-4c72-a09e-d9ce5a10ef9e/sig"

## Unique Gene Names

First we want to select the genes that we wish to look at and format them into a data frame that we can use later on

In [ ]:
#set the directory for outputs
project_ID="PD1_CC_mye_tumor_blood"
scv.settings.figdir=f'{project_ID}_figures'

In [ ]:
#load raw count data
adata = sc.read(f'PD1_CC_mye_anno_raw.h5ad')
adata.raw=adata

In [ ]:
raw=pd.DataFrame(adata.raw.X.toarray(), index=adata.raw.obs_names, columns=adata.raw.var_names)

In [ ]:
adata_raw= sc.AnnData(raw)
for i in np.unique(adata.obs.columns):
    adata_raw.obs[i]=adata.obs[i]
scv.pp.filter_and_normalize(adata_raw, min_cells=0.005*(adata.n_obs), flavor="seurat")

In [ ]:
adata.raw=adata_raw

In [ ]:
Sample_idx=pd.Series("Unknown", index=adata.obs_names)
m=0
for i in np.unique(adata.obs.Patient):   
    Sample_idx[adata.obs.Patient==i]=m
    m=m+1
adata.obs["Sample_idx"]=Sample_idx.values
adata.obs["Sample_idx"]=adata.obs["Sample_idx"].astype(float)

In [ ]:

#we need to make a directory to store the files in
try:
    directoryName=project_ID+"_figures"
    os.mkdir(directoryName)
except:
    pass

#we need to make a directory to store the files in
#we need to make a directory to store the files in
try:
    directoryName=project_ID+"_DEG"
    os.mkdir(directoryName)
except:
    pass


## Preprocess the Data

now we will take the files from the previous section where we determined spliced/unspliced RNA counts and use them to create RNA velocity trajectory as well as investigate biological alternation in each cell

In [ ]:
#calculate the percentage of mitochondrial genes and ribosomal genes
adata.var['MT'] = adata.var_names.str.startswith('MT-')
adata.var['RP'] = adata.var_names.str.startswith('RP')
sc.pp.calculate_qc_metrics(adata, qc_vars=['MT'], percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(adata, qc_vars=['RP'], percent_top=None, log1p=False, inplace=True)
#sc.pl.violin(adata, ['n_vars'], groupby='Sample_ID', stripplot=False) 

In [ ]:
#plot scatter plots of dataset QC
sc.pl.scatter(adata, x='n_genes_by_counts',y='pct_counts_MT',color="Cluster")
sc.pl.scatter(adata, x='n_genes_by_counts', y='pct_counts_RP')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

when cell membrane is broken, cytoplasmic RNA will be easy to leek out, but mitochondia are still too big to pass the broken membrane, so that high percentage usually suggestes bad cells. But on the contrary, there is paper saying stemness like cell containing high percentage mitochondrial genes to survie stress, so we have to go back to this parameters according to what we will have found later.

In [ ]:
sc.pp.filter_genes(adata, min_cells=1,)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5,)
adata = adata[:, adata.var.highly_variable]
scv.tl.score_genes_cell_cycle(adata) # calculate cell cycle score with scv internal function and its gene list of s phase and g2m phase, then we will have two observations---"S_score" and "G2M_score" added in adata 
#sc.pp.regress_out(adata, keys=["total_counts","n_genes_by_counts","Sample_idx"] , n_jobs=None, copy=False) # we remove cell cycle effects
#scv.pp.moments(adata,n_pcs=10, n_neighbors=250, mode="distances") # we calculate a moment matrix for further RNA velocity analysis

In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

In [ ]:
#calculate umap
scv.tl.umap(adata, n_components=2, min_dist=0.7, spread=1, maxiter=None, alpha=1.0,
            gamma=1, negative_sample_rate=5, init_pos="spectral", random_state=0, a=None,
            b=None, copy=False, method="umap", neighbors_key=None)

In [ ]:
#calculate louvain clusters based on the umap we generated previously
sc.tl.leiden(adata, resolution=0.5,
                 key_added='leiden', use_weights=True)
adata.obs["clusters"]=adata.obs["leiden"]

In [ ]:
#check proportions of spliced and unspliced RNA in each cluster
scv.pl.proportions(adata)

In [ ]:
#plot umap grouped by treatment
plot_pattern="anno_sub"
scv.pl.scatter(adata, save=f"{plot_pattern}_umap.pdf",
               basis="umap",
           color=plot_pattern,legend_loc="right",
            size=5, alpha=0.8,
            palette=[ "DeepSkyBlue","Pink","darkviolet",
                    "SpringGreen","forestgreen","maroon"
            ,"PapayaWhip", "gold","orchid","crimson"
            ,"firebrick","olivedrab","lightcoral","greenyellow"
            ,"tan","slategrey","coral","lightseagreen"
            ,"lightsteelblue","lightskyblue","dimgrey",
            "indigo","darkturquoise"], )

In [ ]:
#re-load the processed data
adata = sc.read(f'{project_ID}_figures/{project_ID}_anno_sub.h5ad')

In [ ]:
raw=pd.DataFrame(adata.raw.X.copy(), index=adata.raw.obs_names, columns=adata.raw.var_names)

In [ ]:
#for tumor myeloid subtyping
anno=pd.Series("malign",index=adata.obs_names)
anno[adata.obs.clusters!="X"]="C5AR1_lo"

anno[(((raw["C5AR1"]<np.quantile(raw["C5AR1"],0.25))|(raw["C5AR1"]==np.quantile(raw["C5AR1"],0.25)))
      |((raw["CD163"]<np.quantile(raw["CD163"],0.25))| (raw["CD163"]==np.quantile(raw["CD163"],0.25)))|
     ( (raw["MRC1"]<np.quantile(raw["MRC1"],0.25))|(raw["MRC1"]==np.quantile(raw["MRC1"],0.25))))&(raw["CD86"]>0)]="CD86_hi"
anno[(raw["C5AR1"]>np.quantile(raw["C5AR1"],0.25))]="C5AR1_hi"
adata.obs["anno_sub"]=anno.values

In [ ]:
#for blood myeloid subtyping
anno=pd.Series("malign",index=adata.obs_names)
anno[adata.obs.clusters!="X"]="C5AR1_lo"

anno[(((raw["C5AR1"]<np.quantile(raw["C5AR1"],0.25))|(raw["C5AR1"]==np.quantile(raw["C5AR1"],0.25)))
      |((raw["CD163"]<np.quantile(raw["CD163"],0.25))| (raw["CD163"]==np.quantile(raw["CD163"],0.25)))|
     ( (raw["MRC1"]<np.quantile(raw["MRC1"],0.25))|(raw["MRC1"]==np.quantile(raw["MRC1"],0.25))))&(raw["CD86"]>0)]="CD86_hi"
anno[(raw["C5AR1"]>np.quantile(raw["C5AR1"],0.25))]="C5AR1_hi"
# in blood we spot 10% of the myeloid cell expressed CD163 but C5AR1 low myeloid cells but included in CD86 high group, given CD163 was reported as a M2 marker, they were excluded from CD86 high group.
anno[(raw["CD163"]>np.quantile(raw["CD163"],0.25))&(raw["CD86"]==0)]="C5AR1_lo"

adata.obs["anno_sub"]=anno.values

In [ ]:
#sc.tl.dendrogram(adata, groupby="treatment")
markers ={
    #"Epi.cells":["EPCAM","KRT8","KRT81","RPS19"],
    "Myeloid.cells":["CD14","ITGAM"],
    "Macrophages":["CD68","C1QA","C1QB","C1QC","CSF1R"],
    "M1_TAM":["CD86","CXCL10","CXCL9","CX3CR1"],
    "M2_TAM":["MRC1","CD163","CCL2","C5AR1"],

}
sc.pl.dotplot(adata, markers,standard_scale="var",cmap="RdYlBu_r",figsize=(5,1),
                     mean_only_expressed=False, expression_cutoff=0,
                     groupby='anno_sub', dendrogram=False,save=f"{project_ID}_clusters.pdf")

In [ ]:
#plot umap grouped by treatment
plot_pattern="anno_sub"
scv.pl.scatter(adata, save=f"{plot_pattern}_umap.pdf",
               basis="umap",
           color=plot_pattern,legend_loc="right",
            size=20, alpha=0.8,
            palette=[ "DeepSkyBlue","Pink","darkviolet",
                    "SpringGreen","forestgreen","maroon"
            ,"PapayaWhip", "gold","orchid","crimson"
            ,"firebrick","olivedrab","lightcoral","greenyellow"
            ,"tan","slategrey","coral","lightseagreen"
            ,"lightsteelblue","lightskyblue","dimgrey",
            "indigo","darkturquoise"], )

## Cell fraction bar charts

In [ ]:
adata.obs["Sample"]=adata.obs.Patient.astype(str)+"_"+adata.obs.Group.astype(str)

In [ ]:
adata_s=adata[(adata.obs.Group.str.startswith("Pre"))
              #&((adata.obs.Efficacy=="PR"))
              #|(adata.obs.Efficacy=="PD"))
             #&(adata.obs.Treatment!="Chemo")
              &(adata.obs.Origin=="b")]

In [ ]:
adata_s=adata[(adata.obs.Group.str.startswith("Post"))
              #&((adata.obs.Efficacy=="SD")
              #|(adata.obs.Efficacy=="PD"))
             # &(adata.obs.Treatment!="Chemo")
              &(adata.obs.Origin=="b")]

In [ ]:
#plot umap grouped by treatment
plot_pattern="anno_sub"
scv.pl.scatter(adata, save=f"{plot_pattern}_umap.pdf",
               basis="umap",
           color=plot_pattern,legend_loc="right",
            size=10, alpha=0.8,
            palette=[ "DeepSkyBlue","Pink","darkviolet",
                    "SpringGreen","forestgreen","maroon"
            ,"PapayaWhip", "gold","orchid","crimson"
            ,"firebrick","olivedrab","lightcoral","greenyellow"
            ,"tan","slategrey","coral","lightseagreen"
            ,"lightsteelblue","lightskyblue","dimgrey",
            "indigo","darkturquoise"], )

In [ ]:
# we can plot feature percentage in the population we group cells, treatment or clusters
# first we need to call a cell_fraction matrix containing matched feature and group_by information of each single cell
feature="anno_sub"
group_by="Sample"#treatment Group
cell_fraction=pd.DataFrame(adata_s.obs[group_by].values,columns=[group_by],index=adata_s.obs[feature])

In [ ]:
fraction=pd.DataFrame([])
for i in np.unique(adata_s.obs[group_by]):
    adata_=adata_s[adata_s.obs[group_by]==i]
    fraction_=np.array([])
    for x in np.unique(adata_s.obs[feature]):
        frac=len(adata_[adata_.obs[feature]==x])
        fraction_=pd.Series(np.append(fraction_, frac))
    fraction=pd.concat([fraction, fraction_], axis=1)
fraction.columns=np.unique(adata_s.obs[group_by])
fraction.index=np.unique(adata_s.obs[feature])

In [ ]:
# we calculate the percentage of each feature in each group
fraction.loc["sum"]=fraction.sum(axis=0)
for i in np.unique(adata_s.obs[feature]):
    fraction.loc[f"{i}_pct"]=fraction.loc[i]/fraction.loc["sum"]
fraction.to_csv(f"{project_ID}_figures/{feature}_fraction_{group_by}.csv")

In [ ]:
# we drop the cell number rows and leave the percentage rows
fraction_=fraction.T
for i in np.unique(adata_s.obs[feature]):
    fraction_=fraction_.drop([i],axis=1)
fraction_=fraction_.drop(["sum"],axis=1)
fraction_

In [ ]:
adata.write(f"{project_ID}_figures/{project_ID}_anno_sub.h5ad")